In [ ]:
# Nepal Gorkha 2015 Earthquake Magnitude, Damage, and Impact data
# https://www.kaggle.com/datasets/arashnic/earthquake-magnitude-damage-and-impact
import pandas as pd
import geopandas as gpd
from data_manipulations import * 

# Load the data
buildings = pd.read_csv('./csv_building_structure.csv')
csv_household_demographics = pd.read_csv('./csv_household_demographics.csv')
csv_household_resources = pd.read_csv('./csv_household_resources.csv')
mapping = pd.read_csv('./mapping.csv')

In [ ]:
# Identify new constructions (age <= 10)
buildings['new_construction'] = buildings['age_building'] <= 10

# Calculate household poverty using upper bound on household monthly income range.
# Defining poverty as below 20K rupees / person / year. This ends up being slightly more generous than the UN definition of $1.90 / person / day
# https://databankfiles.worldbank.org/data/download/poverty/987B9C90-CB9F-4D93-AE8C-750588BF00QA/SM2020/Global_POVEQ_NPL.pdf
csv_household_demographics['income_level_household_numeric'] = csv_household_demographics['income_level_household'].apply(numeric_income)
csv_household_demographics['poverty'] = (csv_household_demographics['income_level_household_numeric'] * 12.0 / csv_household_demographics['size_household']) <= 20000.0

# Correlate households and buildings
# buildings = csv_building_structure.merge(csv_building_damage_assessment, on='building_id')
households = csv_household_demographics.merge(mapping[['household_id', 'building_id']].drop_duplicates(), on='household_id')
households = households.merge(buildings, on='building_id')

In [ ]:
# Damage grade for new vs old construction
damage_grade_new_construction = histogram(households, 'damage_grade', 'new_construction')
histogram_plot(damage_grade_new_construction, 'damage_grade', 'new_construction')
damage_grade_new_construction

In [ ]:
# Intensity of building damage by household, poverty
damage_grade = histogram(households, 'damage_grade', 'poverty')
histogram_plot(damage_grade, 'damage_grade', 'poverty')
damage_grade

In [ ]:
# Condition of building after earthquake by household, poverty
condition_post_eq = histogram(households, 'condition_post_eq', 'poverty')
histogram_plot(condition_post_eq, 'condition_post_eq', 'poverty')
condition_post_eq

In [ ]:
# Condition of building after earthquake by household, poverty (female head of household)
condition_post_eq_f = histogram(households[households['gender_household_head'] == 'Female'], 'condition_post_eq', 'poverty')
histogram_plot(condition_post_eq_f, 'condition_post_eq', 'poverty')
condition_post_eq_f

In [ ]:
# Proposed repairs by household, poverty
technical_solution = histogram(households, 'technical_solution_proposed', 'poverty')
histogram_plot(technical_solution, 'technical_solution_proposed', 'poverty')
technical_solution

In [ ]:
# Proposed repairs by household, poverty (female head of household)
technical_solution_f = histogram(households[households['gender_household_head'] == 'Female'], 'technical_solution_proposed', 'poverty')
histogram_plot(technical_solution_f, 'technical_solution_proposed', 'poverty')
technical_solution_f

In [ ]:
condition_post_eq_with_f = condition_post_eq.join(condition_post_eq_f, rsuffix=' Female HOH')
# condition_post_eq_with_f.plot.bar(y=['Poor', 'Not Poor', 'Poor Female HOH', 'Not Poor Female HOH'])
condition_post_eq_with_f.plot.bar(y=['condition_post_eq_poverty_true_percentage', 'condition_post_eq_poverty_false_percentage', 'condition_post_eq_poverty_true_percentage Female HOH', 'condition_post_eq_poverty_false_percentage Female HOH'])


In [ ]:
# Load and clean geo data
geojson = gpd.read_file('./municipality.geojson')
ward_vdcmun_district_name_mapping = pd.read_csv('./ward_vdcmun_district_name_mapping.csv')
ward_vdcmun_district_name_mapping['vdc'] = ward_vdcmun_district_name_mapping['vdcmun_name'].str.replace('Rural Municipality', '')
ward_vdcmun_district_name_mapping['vdc'] = ward_vdcmun_district_name_mapping['vdc'].str.replace('Municipality', '')
ward_vdcmun_district_name_mapping['vdc'] = ward_vdcmun_district_name_mapping['vdc'].str.replace('Sub-Metropolitan City', '')
ward_vdcmun_district_name_mapping['vdc'] = ward_vdcmun_district_name_mapping['vdc'].str.strip()


In [ ]:
vdcs = ward_vdcmun_district_name_mapping[['vdcmun_id', 'vdc', 'district_id', 'district_name']].drop_duplicates()
geojson.merge(vdcs, how='right', left_on='GaPa_NaPa', right_on='vdc')
buildings = buildings.merge(vdcs, left_on='vdcmun_id_x', right_on='vdcmun_id')


In [ ]:
# Choropleth of new construction
buildings['new_construction'] = buildings['age_building'] < 10
buildings
new_construction = buildings[buildings['new_construction']].groupby('vdcmun_id_x').size().to_frame().reset_index()
new_construction.rename(columns={0:'new_construction'}, inplace=True)
new_construction

# geojson2 = geojson.merge(new_construction[['vdcmun_id', 'new_construction']].drop_duplicates(), how='left', left_on="GaPa_NaPa", right_on="vdc")
# geojson2['new_construction'] = geojson2['new_construction'].fillna(0)
# geojson2.plot(column='new_construction')
